In [1]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

# 初始化所需變數

In [2]:
epochs = 10       #訓練的次數
img_rows = None   #驗證碼影像檔的高
img_cols = None   #驗證碼影像檔的寬
digits_in_img = 4 #驗證碼影像檔中有幾位數
x_list = list()   #存所有驗證碼數字影像檔的array
y_list = list()   #存所有的驗證碼數字影像檔array代表的正確數字
x_train = list()  #存訓練用驗證碼數字影像檔的array
y_train = list()  #存訓練用驗證碼數字影像檔array代表的正確數字
x_test = list()   #存測試用驗證碼數字影像檔的array
y_test = list()   #存測試用驗證碼數字影像檔array代表的正確數字

寫一個將驗證碼4位數獨立切出的funciton
驗證碼數字影像檔的array會存在x_list，驗證碼數字影像檔array代表的正確數字會存在y_list

In [3]:
def split_digits_in_img(img_array, x_list, y_list):
    for i in range(digits_in_img):
        step = img_cols // digits_in_img
        x_list.append(img_array[:, i * step:(i + 1) * step] / 255)
        y_list.append(img_filename[i])

從Train_label資料夾以灰階的形式讀入所有.jpg的驗整碼，並逐一將4位數驗證碼影像檔切出

In [4]:
img_filenames = os.listdir('CaptchaJpg\Train_label')
 
for img_filename in img_filenames:
    if '.jpg' not in img_filename:
        continue
    img = load_img(r'C:\Users\dl\Desktop\yc_data\babyCrawler\CaptchaJpg\Train_label\{0}'.format(img_filename), color_mode='grayscale')
    img_array = img_to_array(img)
    img_rows, img_cols, _ = img_array.shape
    split_digits_in_img(img_array, x_list, y_list)


# 除錯

In [5]:
# 找出無效的標籤 ex檔名有誤 大部分是有無效的字串無法轉換為整數 例如出現.
invalid_indices = []
for i, label in enumerate(y_list):
    try:
        int_label = int(label)
    except ValueError:
        invalid_indices.append(i)

# 修正無效標籤
for index in invalid_indices:
    y_list[index] = '0'  # 將無效標籤替換為有效的標籤，例如使用0

# 轉換為數字型標籤
y_list = [int(label) for label in y_list]


In [6]:
img_array.shape

(31, 70, 1)

將y_list所存的驗證碼正確數字0-9轉成categorical形式
舉例: 1 = [0, 1, 0, 0, 0, 0, 0, 0, 0, 0,]、2 = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0,]、依此類推...
然後將所有的資料拆成訓練用及測試用資料集

In [7]:
y_list = keras.utils.to_categorical(y_list, num_classes=10)
x_train, x_test, y_train, y_test = train_test_split(x_list, y_list)

In [8]:
if os.path.isfile('cnn_model.h5'):
    model = models.load_model('cnn_model.h5')
    print('Model loaded from file.') 
else:
    model = models.Sequential()
    model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(img_rows, img_cols // digits_in_img, 1)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(rate=0.25))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(rate=0.5))
    model.add(layers.Dense(10, activation='softmax'))
    print('New model created.')
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),run_eagerly=True, metrics=['accuracy'])

New model created.


In [9]:
x_train = np.squeeze(x_train)
x_test = np.squeeze(x_test)

In [10]:
model.fit(np.array(x_train), np.array(y_train), batch_size=digits_in_img, epochs=epochs, verbose=1, validation_data=(np.array(x_test), np.array(y_test)))

loss, accuracy = model.evaluate(np.array(x_test), np.array(y_test), verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

 
model.save('cnn_model.h5')

Epoch 1/10
702/702 [==============================] - 26s 37ms/step - loss: 1.0290 - accuracy: 0.6585 - val_loss: 0.1907 - val_accuracy: 0.9487
Epoch 2/10
702/702 [==============================] - 26s 36ms/step - loss: 0.2135 - accuracy: 0.9430 - val_loss: 0.1632 - val_accuracy: 0.9626
Epoch 3/10
702/702 [==============================] - 26s 36ms/step - loss: 0.1502 - accuracy: 0.9597 - val_loss: 0.1624 - val_accuracy: 0.9743
Epoch 4/10
702/702 [==============================] - 26s 36ms/step - loss: 0.1281 - accuracy: 0.9658 - val_loss: 0.1775 - val_accuracy: 0.9722
Epoch 5/10
702/702 [==============================] - 25s 36ms/step - loss: 0.0917 - accuracy: 0.9733 - val_loss: 0.1753 - val_accuracy: 0.9743
Epoch 6/10
702/702 [==============================] - 26s 37ms/step - loss: 0.0885 - accuracy: 0.9750 - val_loss: 0.1954 - val_accuracy: 0.9722
Epoch 7/10
702/702 [==============================] - 26s 36ms/step - loss: 0.0658 - accuracy: 0.9825 - val_loss: 0.2119 - val_accuracy: